$ \newcommand{\ket}[1]{|#1\rangle} $

<h1>DAG size estimation</h1>

Here we implement an algorithm for DAG size estimation from paper "Quantum algorithm for tree size estimation, with applications to backtracking and 2-player games" by Andris Ambainis and Martins Kokainis.

We implement it for a complete binary tree with the given number of layers. Additional parameter is remove_pair_count that allows to remove pairs of leaves from the tree. This parameter is used to test the algorithm for different number of edges.

<h3>Parameters</h3>

num_of_layers = distance from the root to farthest leaf, $n$ in paper

bits_of_precision - how many qubits will be used in Phase estimation (eigenvalue estimation), minimal value is $\lceil \log \frac{1}{\delta_{min}} \rceil$

currently, $\delta_{min} = \frac{\delta^{1.5}}{4\sqrt{3nT_0}}$.

$\alpha = \sqrt{2n\delta^{-1}}$

In [1]:
from math import sqrt, log
num_of_layers = 2
bits_of_precision = 9
delta = 0.15
t0 = 2**(num_of_layers+1)
print(t0)
delta_min = delta**1.5/(4*sqrt(3*num_of_layers*t0))
print('number of bits of precision needed:',log(1/delta_min,2))
remove_pair_count = 1
alpha = sqrt(2*num_of_layers/delta)
print('alpha =',alpha)

8
number of bits of precision needed: 8.897929641609888
alpha = 5.163977794943222


<h3>Preparing qubit indecies</h3>

We use '00', '01', and '10' to mark edges and parents/children. Overall, we have a full binary tree with/without 2 neighbor leaves removed.

In [2]:
item_size = 2*num_of_layers
root = ''
for i in range(num_of_layers):
    root += '00'
layers = [[root]]

for cur_layer in range(1,num_of_layers+1):
    layers.append([])
    for i in (layers[cur_layer-1]):
        postfix = i[item_size-(cur_layer-1)*2:]
        prefix = ''
        for j in range(item_size-(cur_layer)*2):
            prefix += '0'
        layers[cur_layer].append(prefix+'01'+postfix)
        layers[cur_layer].append(prefix+'10'+postfix)
psi = {}

print('Initial edge structure')
print(layers)
for i in range(remove_pair_count):
    layers[num_of_layers].pop()
    layers[num_of_layers].pop()
print('Final edge structure')
print(layers)

Initial edge structure
[['0000'], ['0001', '0010'], ['0101', '1001', '0110', '1010']]
Final edge structure
[['0000'], ['0001', '0010'], ['0101', '1001']]


<h3>Preparing the circuit for $\ket{s_{v_1}}$</h3>

In [3]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(2)

sum_of_squares = 1 + 2*(alpha**2)

desired_vector = [
    1/sum_of_squares**0.5, # 00
    alpha/sum_of_squares**0.5, # 01
    alpha/sum_of_squares**0.5, # 10
    0 #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw()

┌───────────────┐ ┌───┐┌───────────────┐┌───┐
q0_0: ─|0>─┤ R(1.3795,π/2) ├─┤ X ├┤ R(1.3795,π/2) ├┤ X ├
           ├───────────────┴┐└─┬─┘└───────────────┘└─┬─┘
q0_1: ─|0>─┤ U3(1.5524,0,0) ├──■─────────────────────■──
           └────────────────┘

In [5]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi

psi[layers[0][0]] = QuantumCircuit(2*num_of_layers+1)
psi[layers[0][0]].cu3(qc2.data[1][0].params[0],0,0,0,1)
psi[layers[0][0]].cu3(qc2.data[3][0].params[0],0,0,0,2)
psi[layers[0][0]].ccx(0,2,1)
psi[layers[0][0]].cu3(qc2.data[5][0].params[0],0,0,0,1)
psi[layers[0][0]].ccx(0,2,1)

psi[layers[0][0]].draw()

q_0: ────────■─────────────────■───────────■──────────■───────────■──
     ┌───────┴────────┐        │         ┌─┴─┐┌───────┴────────┐┌─┴─┐
q_1: ┤ U3(1.3795,0,0) ├────────┼─────────┤ X ├┤ U3(1.3795,0,0) ├┤ X ├
     └────────────────┘┌───────┴────────┐└─┬─┘└────────────────┘└─┬─┘
q_2: ──────────────────┤ U3(1.5524,0,0) ├──■──────────────────────■──
                       └────────────────┘                            
q_3: ────────────────────────────────────────────────────────────────
                                                                     
q_4: ────────────────────────────────────────────────────────────────

<h3>Preparing circuits for other $\ket{s_{v}}$</h3>

In [6]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi

for m in range(1,num_of_layers-1):
    for k in layers[m]:
        psi[k] = QuantumCircuit(num_of_layers*2+1)
        for l in range(num_of_layers*2):
            if k[l] == '1':
                psi[k].cx(0,len(k)-l)
        psi[k].cu3(pi/4,0,0,0,2*m+1)
        psi[k].cu3(1.23095941734,0,0,0,2*m+2)
        psi[k].ccx(2*m+2,0,2*m+1)
        psi[k].cu3(pi/4,0,0,0,2*m+1)
        psi[k].ccx(2*m+2,0,2*m+1)

        
list_of_chageable_vertices = []
for i in range(1,remove_pair_count+1):
    list_of_chageable_vertices.append(-i)
        
for k in layers[num_of_layers-1]:
    psi[k] = QuantumCircuit(num_of_layers*2+1)
    is_changeable_vertex = False
    for i in list_of_chageable_vertices:
        if(k == layers[num_of_layers-1][i]):
            is_changeable_vertex = True
    if(is_changeable_vertex):
        for l in range(len(k)):
            if k[l] == '1':
                psi[k].cx(0,len(k)-l)
    else:
        for l in range(num_of_layers*2):
            if k[l] == '1':
                psi[k].cx(0,len(k)-l)
        psi[k].cu3(pi/4,0,0,0,2*(num_of_layers-1)+1)
        psi[k].cu3(1.23095941734,0,0,0,2*(num_of_layers-1)+2)
        psi[k].ccx(2*(num_of_layers-1)+2,0,2*(num_of_layers-1)+1)
        psi[k].cu3(pi/4,0,0,0,2*(num_of_layers-1)+1)
        psi[k].ccx(2*(num_of_layers-1)+2,0,2*(num_of_layers-1)+1)
        
for k in layers[num_of_layers]:
    psi[k] = QuantumCircuit(num_of_layers*2+1)
    for l in range(len(k)):
        if k[l] == '1':
            psi[k].cx(0,len(k)-l)

<h3>Preparing $D_v$ operators for each vertex</h3>

In [7]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi

identity_minus_state_zero = QuantumCircuit(num_of_layers*2+1)

for i in range(num_of_layers*2):
    identity_minus_state_zero.cx(0,i+1)

control_states = []
for i in range(num_of_layers*2):
    control_states.append(i)

identity_minus_state_zero.h(num_of_layers*2)
identity_minus_state_zero.mct(control_states, num_of_layers*2)
identity_minus_state_zero.h(num_of_layers*2)

for i in range(num_of_layers*2):
    identity_minus_state_zero.cx(0,i+1)

d={}
for j in layers:
    for i in j: 
        d[i] = QuantumCircuit(num_of_layers*2+1)
        d[i] = d[i].compose(psi[i].inverse())
        d[i] = d[i].compose(identity_minus_state_zero)
        d[i] = d[i].compose(psi[i])

<h3>Eigenvalue estimation</h3>

In [8]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sin
from qiskit.circuit.library import QFT
from datetime import datetime
import time

r_a = QuantumCircuit(num_of_layers*2+1)

# root
r_a = r_a.compose(d[layers[0][0]])

for i in range(1,num_of_layers+1):
    if i%2 == 0:
        for j in layers[i]:
            r_a = r_a.compose(d[j])
    
r_b = QuantumCircuit(num_of_layers*2+1)

for i in range(1,num_of_layers+1):
    if i%2 == 1:
        for j in layers[i]:
            r_b = r_b.compose(d[j])

rbra = r_a
rbra = rbra.compose(r_b)

controlled_rbra = rbra
    
phase_estimation = QuantumCircuit(num_of_layers*2+bits_of_precision,bits_of_precision)
phase_estimation.h(range(bits_of_precision))

operation_qubits = []
for i in range(bits_of_precision,num_of_layers*2+bits_of_precision):
    operation_qubits = operation_qubits+[i]

qubits_to_appply = []
qubits_to_appply += [0]
qubits_to_appply += operation_qubits
for j in range(1):
    phase_estimation = phase_estimation.compose(controlled_rbra,qubits_to_appply)

for i in range(1, bits_of_precision):
    qubits_to_appply = []
    qubits_to_appply += [i]
    qubits_to_appply += operation_qubits
    controlled_rbra = controlled_rbra.compose(controlled_rbra)
    phase_estimation = phase_estimation.compose(controlled_rbra,qubits_to_appply)

phase_estimation.barrier()

phase_estimation = phase_estimation.compose(QFT(num_qubits=bits_of_precision, approximation_degree=0, do_swaps=True, inverse=True, insert_barriers=False, name='qft'),range(bits_of_precision))

phase_estimation.measure(range(bits_of_precision),range(bits_of_precision))
    
job = execute(phase_estimation,Aer.get_backend('qasm_simulator'),shots=10000)
counts = job.result().get_counts(phase_estimation)

ordered_counts = dict(sorted(counts.items(), key=lambda item: item[1],reverse=True))

print(ordered_counts)
print(list(ordered_counts)[0])
print(ordered_counts[list(ordered_counts)[0]])

{'000010000': 2989, '111110000': 2972, '000001111': 1117, '111110001': 1063, '000010001': 238, '111101111': 232, '000001110': 177, '111110010': 152, '000010010': 79, '111101110': 74, '010011100': 69, '000001101': 68, '111110011': 64, '101100100': 63, '000010011': 44, '111101101': 41, '000001100': 41, '111110100': 40, '000010100': 31, '111110101': 28, '000001011': 22, '111101100': 21, '111110110': 16, '111101011': 16, '000001010': 14, '111101010': 14, '111110111': 14, '111100111': 13, '000010101': 13, '000001000': 12, '111101001': 11, '000010110': 11, '000000111': 11, '000001001': 11, '111100110': 9, '000010111': 9, '101100011': 7, '111100101': 7, '000000110': 7, '111111011': 6, '111111110': 6, '000011010': 5, '111111101': 5, '000011000': 5, '000000101': 5, '111111000': 5, '111101000': 5, '000011001': 5, '111111001': 4, '010011011': 4, '111100100': 4, '000000001': 4, '111100001': 3, '111011100': 3, '111100010': 3, '101100101': 3, '000100100': 3, '111011011': 3, '000011100': 3, '11111110

<h3>Converting algorithm outcome to decimal value</h3>

In [9]:
decimal_bits = int(list(ordered_counts)[0], 2)
print(decimal_bits)

16


<h3>Computing value T</h3>

$\theta = 2*\pi*\frac{decimal\_bits}{2^{bits\_of\_precision}}$

$T=\frac{1}{\alpha^2 sin^2 \frac{\theta}{2}}$

In [10]:
theta = 2*pi*decimal_bits/(2**bits_of_precision)
print('theta:',theta)
result = 1/(alpha*alpha*(sin(theta/2)**2))
print('sin:',sin(pi*decimal_bits/(2**bits_of_precision)))
print('result:',result)

theta: 0.19634954084936207
sin: 0.0980171403295606
result: 3.9032575844931547
